In [1]:
%load_ext tensorboard
%load_ext autoreload
%autoreload 2

In [2]:
!pip install 'temporal_fusion_transformer[training] @ git+https://github.com/aaarrti/temporal_fusion_transformer.git@develop'
!pip install "jax[cuda12_pip]" -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html

  Cloning https://github.com/aaarrti/temporal_fusion_transformer.git (to revision develop) to /tmp/pip-install-iklpjdyf/temporal-fusion-transformer_b5bdf9c7e67c442e905c77422261632b
  Running command git clone --filter=blob:none --quiet https://github.com/aaarrti/temporal_fusion_transformer.git /tmp/pip-install-iklpjdyf/temporal-fusion-transformer_b5bdf9c7e67c442e905c77422261632b
  Running command git checkout -b develop --track origin/develop
  Switched to a new branch 'develop'
  Branch 'develop' set up to track remote branch 'develop' from 'origin'.
  Resolved https://github.com/aaarrti/temporal_fusion_transformer.git to commit 0e623c9255d80b8607853f3c8237fff6aad76b9a
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 8.8 MB

First, we'd need to upload dataset to GDrive

In [3]:
from google.colab import drive

drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
import tensorflow as tf
import jax
from absl_extra import logging_utils
import temporal_fusion_transformer as tft
from temporal_fusion_transformer.src.training_lib import make_training_hooks
import clu.periodic_actions

tf.config.set_visible_devices([], "GPU")
logging_utils.setup_logging(log_level="INFO")
jax.devices()

[gpu(id=0)]

In [3]:
config = tft.get_config("electricity")
config

fixed_params:
  input_known_categorical_idx:
  - 3
  - 4
  - 5
  - 6
  input_known_real_idx:
  - 1
  - 2
  input_observed_idx: []
  input_static_idx:
  - 0
  known_categories_sizes:
  - 31
  - 24
  - 7
  - 12
  num_encoder_steps: 168
  num_outputs: 1
  static_categories_sizes:
  - 369
  total_time_steps: 192
hyperparams:
  dropout_rate: 0.1
  latent_dim: 160
  num_attention_heads: 4
  num_decoder_blocks: 1
  quantiles:
  - 0.1
  - 0.5
  - 0.9
optimizer:
  clipnorm: 0.0
  decay_alpha: 0.05
  decay_steps: 0.0
  ema: 0.0
  learning_rate: 0.0005
prng_seed: 69
shuffle_buffer_size: 1024

Run training

In [4]:
batch_size = 128
total_samples = 1853056

hooks = make_training_hooks(
    total_samples // batch_size,
    epochs=1,
    add_early_stopping=False,
    checkpoint_directory=None,
    logdir="tensorboard",
)
profiler = clu.periodic_actions.Profile(
    logdir="/content/drive/MyDrive/tensorboard/electricity",
    every_steps=200,
)


def call_profiler(step: int, **kwargs):
    profiler(step)
    hooks.on_step_begin.append(call_profiler)


hooks.on_step_begin.append(call_profiler)

In [ ]:
tft.training_scripts.train_experiment_on_single_device(
    data_dir="/content/drive/MyDrive/datasets",
    batch_size=128,
    experiment_name="electricity",
    mixed_precision=True,
    config=config,
    jit_module=True,
    hooks=hooks,
    prefetch_buffer_size=2,
)

INFO:absl:Epoch 1/1...


    1/14477 [..............................] - ETA: 14:48

INFO:absl:[1] collection=training loss=50.15625


    6/14477 [..............................] - ETA: 48:30:00